# 금융상품 추천 시스템 개발(CatBoost)

---

## 🎯 프로젝트 개요

### 목적
개인의 특성(나이, 소득, 투자성향 등)을 분석하여 적합한 금융상품을 AI로 추천하는 시스템

### 주요 기능
- **다중 라벨 분류**: 사용자에게 여러 금융상품 동시 추천
- **실제 상품 연동**: 금융감독원 API 데이터와 연결된 실제 상품 정보
- **대화형 입력**: 사용자 친화적인 정보 입력 인터페이스
- **카테고리별 추천**: 예금, 적금, 대출 등 특정 상품군 맞춤 추천

### 사용 기술
- **머신러닝**: RandomForest + MultiOutputClassifier
- **데이터**: 금융감독원 금융상품 통합 비교공시 API
- **평가 지표**: Accuracy, Precision, Recall, F1-Score

---

## 🏗️ 시스템 아키텍처
```
[사용자 입력] → [AI 모델 예측] → [상품 매칭] → [추천 결과]
↓              ↓              ↓            ↓
16개 특성      5개 라벨 확률    JSON 상품DB    맞춤형 추천

### 데이터 흐름
1. **입력**: 사용자의 16개 특성 (나이, 소득, 위험성향 등)
2. **예측**: AI 모델이 5개 금융상품 보유 확률 계산
3. **매칭**: 높은 확률의 카테고리에서 실제 상품 검색
4. **출력**: 금리, 은행명, 가입방법 등 상세 추천


### JSON 매칭 과정 설명

1.  AI 모델이 예측하는 것
AI 모델은 사용자 정보를 보고 **"이 사람이 어떤 금융상품을 보유할 확률"**을 예측합니다.
```
사용자 입력 → AI 모델 → 예측 결과
30세, 5000만원 소득... → 분석 → LIQ: 85%, STOCKS: 20%, CDS: 5%...
```
2.  예측 결과를 상품 카테고리로 변환
AI가 예측한 **라벨(LIQ, STOCKS 등)**을 실제 JSON 파일명으로 연결합니다.
```
pythonLABEL_TO_CATEGORY = {
    'LIQ': 'bank_deposits',      # 유동성자산 → 예금상품 JSON
    'CDS': 'bank_deposits',      # 양도성예금증서 → 예금상품 JSON  
    'NMMF': 'bank_savings',      # 비머니마켓펀드 → 적금상품 JSON
    'STOCKS': 'credit_loans',    # 주식보유 → 신용대출 JSON (투자자금)
    'RETQLIQ': 'bank_savings'    # 퇴직준비금 → 적금상품 JSON
}
예시: AI가 "LIQ 85% 확률"이라고 예측하면 → bank_deposits.json 파일을 열어봅니다.
```
3.  JSON에서 실제 상품 찾기
```
해당 JSON 파일을 열어서 금리가 높은 순서로 실제 상품들을 찾습니다.
bank_deposits.json 열기 → 모든 예금상품 리스트 → 금리 높은 순 정렬 → 상위 3개 선택
```
4.  사용자에게 보여주기
```
AI 예측 + 실제 상품 정보를 합쳐서 사용자에게 추천합니다.
최종 결과:
📂 유동성자산 (85% 확률) ✅ 추천
  1. Sh첫만남우대예금 (수협은행, 2.90%)
  2. NH고향사랑기부예금 (농협은행, 2.80%)
```

### 전체 흐름 요약
```
사용자 입력 → 나이, 소득 등
AI 예측 → LIQ 85%, STOCKS 20% 등
매핑 테이블 → LIQ = bank_deposits.json
JSON 검색 → 예금상품들 중 금리 높은 것들
최종 추천 → 구체적인 은행 상품명과 금리

핵심은 AI가 "어떤 종류의 상품이 적합한지" 예측하면, 그 종류에 해당하는 실제 상품들을 JSON에서 찾아서 보여주는 것입니다! 🎯
```


## 💻 코드 구조 및 설명

### 1. 환경 설정 및 라이브러리
**설명**: 데이터 처리, 머신러닝 모델, 성능 평가에 필요한 라이브러리들을 불러옵니다.

In [1]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
import warnings
import pandas as pd
from sklearn.multioutput import MultiOutputClassifier
import time
warnings.filterwarnings('ignore')

### 2. 상품 카테고리 정의
**설명**: 
- AI 모델의 예측 라벨과 실제 상품 카테고리를 연결하는 매핑 테이블
- JSON 파일 경로와 상품 설명 정의

In [2]:
# 2: 상품 카테고리 정의
PRODUCT_CATEGORIES = {
    'bank_deposits': {
        'name': '예금상품', 
        'file': '../1.데이터수집/data/bank_deposits.json',
        'description': '정기예금, 자유적금 등'
    },
    'bank_savings': {
        'name': '적금상품', 
        'file': '../1.데이터수집/data/bank_savings.json',
        'description': '정기적금, 자유적금 등'
    },
    'credit_loans': {
        'name': '개인신용대출', 
        'file': '../1.데이터수집/data/credit_loans.json',
        'description': '신용대출, 마이너스통장 등'
    },
    'mortgage_loans': {
        'name': '주택담보대출', 
        'file': '../1.데이터수집/data/mortgage_loans.json',
        'description': '주택구입자금, 주택개량자금 등'
    },
    'rent_loans': {
        'name': '전세자금대출', 
        'file': '../1.데이터수집/data/rent_loans.json',
        'description': '전세자금, 보증금 대출 등'
    }
}

# 모델 라벨과 상품 카테고리 매핑
LABEL_TO_CATEGORY = {
    'MMMF': 'bank_deposits',      # 유동성자산 → 예금
    'CDS': 'bank_deposits',      # 양도성예금증서 → 예금  
    'NMMF': 'bank_savings',      # 비머니마켓펀드 → 적금
    'STOCKS': 'credit_loans',    # 주식보유 → 신용대출 (투자자금)
    'RETQLIQ': 'bank_savings'    # 퇴직준비금유동성 → 적금
}

target_names = {
    'MMMF': '단기금융상품펀드',
    'CDS': '양도성예금증서', 
    'NMMF': '비머니마켓펀드',
    'STOCKS': '주식보유',
    'RETQLIQ': '퇴직준비금유동성'
}

### 3. AI 모델 훈련 
**주요 특징**:
- **RandomForest**: 여러 의사결정트리의 앙상블로 과적합 방지
- **MultiOutputClassifier**: 5개 상품을 동시에 예측하는 다중 라벨 분류
- **class_weight='balanced'**: 데이터 불균형 문제 자동 해결

#### 📊 성능 평가

##### 평가 지표 해석

**1. 정확도 (Accuracy)**
- **의미**: 전체 예측 중 맞춘 비율
- **목표**: 60-75% (금융 추천에서는 양호)
- **해석**: 70% = 10명 중 7명에게 정확한 추천

**2. 정밀도 (Precision)**
- **의미**: 추천한 상품 중 실제로 적합한 비율
- **목표**: 80% 이상
- **해석**: 80% = 추천 상품 10개 중 8개가 정확

**3. 재현율 (Recall)**
- **의미**: 실제 적합한 상품 중 놓치지 않고 찾아낸 비율
- **목표**: 70% 이상
- **해석**: 70% = 적합한 상품 10개 중 7개 추천

**4. F1 점수**
- **의미**: 정밀도와 재현율의 조화평균
- **목표**: 75% 이상
- **해석**: 균형잡힌 성능 지표


In [8]:
print("CatBoost 모델 훈련 시작...")

#데이터 로드
df = pd.read_csv('../2.전처리/data/SCFP/cleaned_scf_data.csv')
X = df.iloc[:,:-5]  # 14개 독립변수
y = df.iloc[:,-5:]  # 5개 타겟변수

print(f"데이터 크기: {X.shape[0]:,}명, 특성: {X.shape[1]}개, 타겟: {y.shape[1]}개")

CatBoost 모델 훈련 시작...
데이터 크기: 22,975명, 특성: 14개, 타겟: 5개


In [9]:
# 범주형 특성 식별 (데이터에 맞게 조정 필요)
# 실제 데이터의 범주형 컬럼 인덱스를 확인하여 수정해야 합니다
# 예시: 만약 1, 3, 5번째 컬럼이 범주형이라면
categorical_features = []  # 범주형 특성이 있다면 인덱스 리스트로 지정
# 예: categorical_features = [0, 2, 4]  # 범주형 컬럼의 인덱스

# 범주형 특성이 있는지 자동으로 확인
for i, col in enumerate(X.columns):
    if X[col].dtype == 'object' or X[col].nunique() < 10:  # 범주형 판단 기준
        categorical_features.append(i)

if len(categorical_features) > 0:
    print(f"범주형 특성 발견: {len(categorical_features)}개")
else:
    print("범주형 특성 없음 - 모든 특성을 수치형으로 처리")

범주형 특성 발견: 8개


In [10]:
# 데이터 분할
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

### 🐱 CatBoost 파라미터

- **`iterations=400`** : 다중라벨 × 22K 데이터에 충분한 그래디언트 부스팅 반복
- **`depth=6`** : 64개 리프 노드로 금융 데이터 복잡 패턴 학습
- **`learning_rate=0.1`** : 안정적 수렴과 효율적 훈련의 균형점
- **`l2_leaf_reg=2`** : 금융 데이터 노이즈 대응 + 과적합 방지 정규화
- **`auto_class_weights='Balanced'`** : 각 타겟별 불균형 자동 해결
- **`cat_features=자동감지`** : 범주형 특성 고급 처리로 성능 향상
- **`thread_count=-1`** : 모든 CPU 코어 활용으로 훈련 시간 최소화
- **`allow_writing_files=False`** : 불필요한 로그 파일 생성 방지

In [11]:

%%time
# 다중라벨 CatBoost 모델
base_model = CatBoostClassifier(
    iterations=400,              # 충분한 학습 (2만+ 데이터)
    depth=6,                    # 14개 특성 → depth 5가 적절
    learning_rate=0.1,         # 안정적 학습률
    l2_leaf_reg=2,             # 과적합 방지 (금융 데이터 특성상)
    auto_class_weights='Balanced',  # 불균형 데이터 대응
    # 범주형 특성 설정 ★★★ 중요 ★★★
    cat_features=categorical_features if len(categorical_features) > 0 else None,
    random_seed=42,
    verbose=200,
    thread_count=-1,              # 병렬 처리
    allow_writing_files=False
)

multilabel_model = MultiOutputClassifier(base_model)
multilabel_model.fit(train_X, train_y)

# 모델 성능 평가
test_pred = multilabel_model.predict(test_X)

# 전체 성능 지표
accuracy = accuracy_score(test_y, test_pred)
precision = precision_score(test_y, test_pred, average='samples')
recall = recall_score(test_y, test_pred, average='samples')
f1 = f1_score(test_y, test_pred, average='samples')

print("★ CatBoost 모델 훈련 완료!")
print("\n★ 전체 모델 성능")
print("=" * 40)
print(f"정확도 (Accuracy):  {accuracy:.3f}")
print(f"정밀도 (Precision): {precision:.3f}")
print(f"재현율 (Recall):    {recall:.3f}")
print(f"F1 점수:           {f1:.3f}")

# 개별 상품별 성능
print("\n★ 상품별 성능 지표")
print("=" * 40)

target_names_list = ['단기금융상품펀드', '양도성예금증서', '비머니마켓펀드', '주식보유', '퇴직준비금유동성']

for i, name in enumerate(target_names_list):
    y_true_i = test_y.iloc[:, i]
    y_pred_i = test_pred[:, i]
    
    acc_i = accuracy_score(y_true_i, y_pred_i)
    prec_i = precision_score(y_true_i, y_pred_i, zero_division=0)
    rec_i = recall_score(y_true_i, y_pred_i, zero_division=0)
    f1_i = f1_score(y_true_i, y_pred_i, zero_division=0)
    
    print(f"\n● {name}")
    print(f"   정확도: {acc_i:.3f} | 정밀도: {prec_i:.3f} | 재현율: {rec_i:.3f} | F1: {f1_i:.3f}")

# 성능 해석
print("\n★ 성능 해석")
print("=" * 40)
if accuracy >= 0.7:
    print("★ 우수한 전체 정확도!")
elif accuracy >= 0.6:
    print("◎ 양호한 전체 정확도")
else:
    print("※ 정확도 개선 필요")

if precision >= 0.8:
    print("★ 높은 정밀도 - 추천 신뢰성 우수!")
elif precision >= 0.7:
    print("◎ 양호한 정밀도")
else:
    print("※ 정밀도 개선 필요 - 잘못된 추천 많음")

if recall >= 0.7:
    print("★ 높은 재현율 - 놓치는 상품 적음!")
elif recall >= 0.6:
    print("◎ 양호한 재현율")
else:
    print("※ 재현율 개선 필요 - 놓치는 상품 많음")

0:	learn: 0.6518858	total: 55.8ms	remaining: 22.2s
200:	learn: 0.1647793	total: 9.86s	remaining: 9.76s
399:	learn: 0.0637421	total: 20.9s	remaining: 0us
0:	learn: 0.6701881	total: 53.3ms	remaining: 21.3s
200:	learn: 0.2934562	total: 10.3s	remaining: 10.2s
399:	learn: 0.1573718	total: 20.9s	remaining: 0us
0:	learn: 0.6411463	total: 50.2ms	remaining: 20s
200:	learn: 0.2791139	total: 10s	remaining: 9.92s
399:	learn: 0.1875608	total: 21.9s	remaining: 0us
0:	learn: 0.6494112	total: 54.9ms	remaining: 21.9s
200:	learn: 0.3493341	total: 11.5s	remaining: 11.4s
399:	learn: 0.2656382	total: 22.8s	remaining: 0us
0:	learn: 0.6480336	total: 55.1ms	remaining: 22s
200:	learn: 0.3226838	total: 11.4s	remaining: 11.2s
399:	learn: 0.2555590	total: 22.8s	remaining: 0us
★ CatBoost 모델 훈련 완료!

★ 전체 모델 성능
정확도 (Accuracy):  0.677
정밀도 (Precision): 0.521
재현율 (Recall):    0.578
F1 점수:           0.535

★ 상품별 성능 지표

● 단기금융상품펀드
   정확도: 0.980 | 정밀도: 0.692 | 재현율: 0.943 | F1: 0.798

● 양도성예금증서
   정확도: 0.937 | 정밀도: 0.562 |

In [12]:
# 모델 저장
import joblib
joblib.dump(multilabel_model, 'Models/CatBoost.joblib')

['Models/CatBoost.joblib']